In [2]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import netCDF4 as nc
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeature
import numpy as np
#import pandas as pd
import itertools
#from os import path
import math
import pandas as pd
import xarray as xr
import glob
import wrf
from wrf import getvar, interplevel,interpz3d

In [7]:
#calculate the LWC along IOP22-UWKA flight path.

#read in wrf output files
wrfout_dir='/glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/'
wrfout_filelist=glob.glob(wrfout_dir+'wrfout_*')
ntotal=0
for wrfout_file in wrfout_filelist:
    if int(wrfout_file[97:99])>=14 and int(wrfout_file[97:99])<17:
        ntotal=ntotal+1
print('total number of files=',ntotal)

nfile=0
print('Read wrfout files...')
for wrfout_file in wrfout_filelist:
    if int(wrfout_file[97:99])>=14 and int(wrfout_file[97:99])<17:
        ncfile = Dataset(wrfout_file,mode='r')
        print('read',wrfout_file)
        if nfile==0:
            #calculate the grid index of the sw & ne corner of the selected region
            x_min=-116.60
            x_max=-115.28
            y_min=44.11
            y_max=44.3
            xy_sw1=wrf.ll_to_xy(ncfile,y_min,x_min).data
            xy_ne1=wrf.ll_to_xy(ncfile,y_max,x_max).data
            x_min=-116.8
            x_max=-116.2
            y_min=43.45
            y_max=44.21
            xy_sw2=wrf.ll_to_xy(ncfile,y_min,x_min).data
            xy_ne2=wrf.ll_to_xy(ncfile,y_max,x_max).data
        p = getvar(ncfile, "pressure")
        ht = getvar(ncfile, "z", msl=True,units="m")
        tk = getvar(ncfile,"tk")
        rho=p*100/(283.14*tk)
        qncloud=getvar(ncfile,"QNCLOUD")
        qcloud=getvar(ncfile,"QCLOUD")*1e3 #lwc g/kg
        ncloud=qncloud*rho/1e6
        #ht_500 = interpz3d(ht, p, 500.0)
        #interpolate to height MSL
        qcloud_4876 = interplevel(qcloud, ht, 4876.0)#16,000feet
        qcloud_4572 = interplevel(qcloud, ht, 4572.0)
        ncloud_4876 = interplevel(ncloud, ht, 4876.0)#16,000feet
        ncloud_4572 = interplevel(ncloud, ht, 4572.0)

        #get average droplet number concentration over the region
        ncloud_region1=ncloud_4876[xy_sw1[1]-1:xy_ne1[1]+1,xy_sw1[0]-1:xy_ne1[0]+1].data
        ncloud_region1=ncloud_region1.reshape(ncloud_region1.size,)
        ncloud_region2=ncloud_4572[xy_sw1[1]-1:xy_ne1[1]+1,xy_sw1[0]-1:xy_ne1[0]+1].data
        ncloud_region2=ncloud_region2.reshape(ncloud_region2.size,)
        ncloud_region3=ncloud_4572[xy_sw2[1]-1:xy_ne2[1]+1,xy_sw2[0]-1:xy_ne2[0]+1].data
        ncloud_region3=ncloud_region3.reshape(ncloud_region3.size,)
        ncloud_regions=np.hstack((ncloud_region1,ncloud_region2,ncloud_region3))
        
        
        #get LWC over the region
        qcloud_region1=qcloud_4876[xy_sw1[1]-1:xy_ne1[1]+1,xy_sw1[0]-1:xy_ne1[0]+1].data
        qcloud_region1=qcloud_region1.reshape(qcloud_region1.size,)
        qcloud_region2=qcloud_4572[xy_sw1[1]-1:xy_ne1[1]+1,xy_sw1[0]-1:xy_ne1[0]+1].data
        qcloud_region2=qcloud_region2.reshape(qcloud_region2.size,)
        qcloud_region3=qcloud_4572[xy_sw2[1]-1:xy_ne2[1]+1,xy_sw2[0]-1:xy_ne2[0]+1].data
        qcloud_region3=qcloud_region3.reshape(qcloud_region3.size,)
        qcloud_regions=np.hstack((qcloud_region1,qcloud_region2,qcloud_region3))
        
        #use lower limit N>3.0/cc to filter out the datapoint
        cdnc_min=3.0
        non_zero_ncloud1=ncloud_region1[ncloud_region1>=cdnc_min]
        non_zero_ncloud2=ncloud_region2[ncloud_region2>=cdnc_min]
        non_zero_ncloud3=ncloud_region3[ncloud_region3>=cdnc_min]
        non_zero_nclouds=ncloud_regions[ncloud_regions>=cdnc_min]
        
        positive_qcloud1=qcloud_region1[ncloud_region1>=cdnc_min]
        positive_qcloud2=qcloud_region2[ncloud_region2>=cdnc_min]
        positive_qcloud3=qcloud_region3[ncloud_region3>=cdnc_min]
        positive_qclouds=qcloud_regions[ncloud_regions>=cdnc_min]
        
        if nfile == 0:
            NCLOUD_R1_series=non_zero_ncloud1
            NCLOUD_R2_series=non_zero_ncloud2
            NCLOUD_R3_series=non_zero_ncloud3
            NCLOUD_ALL_series=non_zero_nclouds
            
            QCLOUD_R1_series=positive_qcloud1
            QCLOUD_R2_series=positive_qcloud2
            QCLOUD_R3_series=positive_qcloud3
            QCLOUD_ALL_series=positive_qclouds
        else:
            NCLOUD_R1_series=np.append(NCLOUD_R1_series,non_zero_ncloud1)
            NCLOUD_R2_series=np.append(NCLOUD_R2_series,non_zero_ncloud2)
            NCLOUD_R3_series=np.append(NCLOUD_R3_series,non_zero_ncloud3)
            NCLOUD_ALL_series=np.append(NCLOUD_ALL_series,non_zero_nclouds)
            
            QCLOUD_R1_series=np.append(QCLOUD_R1_series,positive_qcloud1)
            QCLOUD_R2_series=np.append(QCLOUD_R2_series,positive_qcloud2)
            QCLOUD_R3_series=np.append(QCLOUD_R3_series,positive_qcloud3)
            QCLOUD_ALL_series=np.append(QCLOUD_ALL_series,positive_qclouds)
        #np.nonzero(ncloud_select_region)
        nfile=nfile+1
        print('processed ',nfile,' out of',ntotal,'  files')
print('Finished')

total number of files= 18
Read wrfout files...
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_16:20:00
processed  1  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_14:40:00
processed  2  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_15:30:00
processed  3  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_14:30:00
processed  4  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_16:50:00
processed  5  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017-03-09_15:40:00
processed  6  out of 18   files
read /glade/u/home/sisichen/scratch/WRF_SNOWIE/ERA5_run/wrfout_900m_originalCCN/wrfout_d01_2017

In [8]:
#calculate statistics of LWC over the three selected regions
print('calculate the all three region average')
positive_qcloud=QCLOUD_ALL_series#np.append(positive_qcloud1,positive_qcloud2)
max_qcloud=positive_qcloud.max()
mean_qcloud=positive_qcloud.mean()
min_qcloud=positive_qcloud.min()
n5=np.percentile(positive_qcloud, 5) 
n25=np.percentile(positive_qcloud, 25) 
n50=np.percentile(positive_qcloud, 50)
n75=np.percentile(positive_qcloud, 75)
n95=np.percentile(positive_qcloud, 95) 
print('min, max, mean')
print(min_qcloud,max_qcloud,mean_qcloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 1 average')
positive_qcloud=QCLOUD_R1_series#np.append(positive_qcloud1,positive_qcloud2)
max_qcloud=positive_qcloud.max()
mean_qcloud=positive_qcloud.mean()
min_qcloud=positive_qcloud.min()
n5=np.percentile(positive_qcloud, 5) 
n25=np.percentile(positive_qcloud, 25) 
n50=np.percentile(positive_qcloud, 50)
n75=np.percentile(positive_qcloud, 75)
n95=np.percentile(positive_qcloud, 95) 
print('min, max, mean')
print(min_qcloud,max_qcloud,mean_qcloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 2 average')
positive_qcloud=QCLOUD_R2_series#np.append(positive_qcloud1,positive_qcloud2)
max_qcloud=positive_qcloud.max()
mean_qcloud=positive_qcloud.mean()
min_qcloud=positive_qcloud.min()
n5=np.percentile(positive_qcloud, 5) 
n25=np.percentile(positive_qcloud, 25) 
n50=np.percentile(positive_qcloud, 50)
n75=np.percentile(positive_qcloud, 75)
n95=np.percentile(positive_qcloud, 95) 
print('min, max, mean')
print(min_qcloud,max_qcloud,mean_qcloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 3 average')
positive_qcloud=QCLOUD_R3_series#np.append(positive_qcloud1,positive_qcloud2)
max_qcloud=positive_qcloud.max()
mean_qcloud=positive_qcloud.mean()
min_qcloud=positive_qcloud.min()
n5=np.percentile(positive_qcloud, 5) 
n25=np.percentile(positive_qcloud, 25) 
n50=np.percentile(positive_qcloud, 50)
n75=np.percentile(positive_qcloud, 75)
n95=np.percentile(positive_qcloud, 95) 
print('min, max, mean')
print(min_qcloud,max_qcloud,mean_qcloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

calculate the all three region average
min, max, mean
1.985517e-06 1.8050984 0.3085518
5%, 25%, 50%, 75%, 95%
0.0072374336887151 0.060203658416867256 0.19568267464637756 0.4945140928030014 0.9241058230400085
calculate the region 1 average
min, max, mean
3.1307074e-05 0.7253451 0.07488703
5%, 25%, 50%, 75%, 95%
0.0018006806261837483 0.012755841948091984 0.0394972562789917 0.11041206866502762 0.2576370060443878
calculate the region 2 average
min, max, mean
1.985517e-06 1.6473434 0.14046365
5%, 25%, 50%, 75%, 95%
0.0030386420898139478 0.024567517917603254 0.081798255443573 0.19612844288349152 0.4702719256281853
calculate the region 3 average
min, max, mean
1.985517e-06 1.8050984 0.36529878
5%, 25%, 50%, 75%, 95%
0.012270704377442598 0.08901107311248779 0.2749287039041519 0.5886396914720535 0.9662499874830246


In [9]:
#calculate statistics of CDNC over the three selected regions
print('calculate the all three region average')
non_zero_ncloud=NCLOUD_ALL_series#np.append(non_zero_ncloud1,non_zero_ncloud2)
max_ncloud=non_zero_ncloud.max()
mean_ncloud=non_zero_ncloud.mean()
min_ncloud=non_zero_ncloud.min()
n5=np.percentile(non_zero_ncloud, 5) 
n25=np.percentile(non_zero_ncloud, 25) 
n50=np.percentile(non_zero_ncloud, 50)
n75=np.percentile(non_zero_ncloud, 75)
n95=np.percentile(non_zero_ncloud, 95) 
print('min, max, mean')
print(min_ncloud,max_ncloud,mean_ncloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 1 average')
non_zero_ncloud=NCLOUD_R1_series#np.append(non_zero_ncloud1,non_zero_ncloud2)
max_ncloud=non_zero_ncloud.max()
mean_ncloud=non_zero_ncloud.mean()
min_ncloud=non_zero_ncloud.min()
n5=np.percentile(non_zero_ncloud, 5) 
n25=np.percentile(non_zero_ncloud, 25) 
n50=np.percentile(non_zero_ncloud, 50)
n75=np.percentile(non_zero_ncloud, 75)
n95=np.percentile(non_zero_ncloud, 95) 
print('min, max, mean')
print(min_ncloud,max_ncloud,mean_ncloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 2 average')
non_zero_ncloud=NCLOUD_R2_series#np.append(non_zero_ncloud1,non_zero_ncloud2)
max_ncloud=non_zero_ncloud.max()
mean_ncloud=non_zero_ncloud.mean()
min_ncloud=non_zero_ncloud.min()
n5=np.percentile(non_zero_ncloud, 5) 
n25=np.percentile(non_zero_ncloud, 25) 
n50=np.percentile(non_zero_ncloud, 50)
n75=np.percentile(non_zero_ncloud, 75)
n95=np.percentile(non_zero_ncloud, 95) 
print('min, max, mean')
print(min_ncloud,max_ncloud,mean_ncloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

print('calculate the region 3 average')
non_zero_ncloud=NCLOUD_R3_series#np.append(non_zero_ncloud1,non_zero_ncloud2)
max_ncloud=non_zero_ncloud.max()
mean_ncloud=non_zero_ncloud.mean()
min_ncloud=non_zero_ncloud.min()
n5=np.percentile(non_zero_ncloud, 5) 
n25=np.percentile(non_zero_ncloud, 25) 
n50=np.percentile(non_zero_ncloud, 50)
n75=np.percentile(non_zero_ncloud, 75)
n95=np.percentile(non_zero_ncloud, 95) 
print('min, max, mean')
print(min_ncloud,max_ncloud,mean_ncloud)
print('5%, 25%, 50%, 75%, 95%')
print(n5,n25,n50,n75,n95)

calculate the all three region average
min, max, mean
3.0004716 391.5277 117.08858
5%, 25%, 50%, 75%, 95%
17.197097778320312 66.4875717163086 120.0928726196289 165.47293853759766 214.36671447753906
calculate the region 1 average
min, max, mean
3.0278945 391.5277 100.98359
5%, 25%, 50%, 75%, 95%
8.228589630126953 33.028900146484375 81.32849884033203 159.7720184326172 246.35758361816403
calculate the region 2 average
min, max, mean
3.0053496 310.8012 128.70049
5%, 25%, 50%, 75%, 95%
12.594487571716307 57.41620445251465 131.01080322265625 193.62057876586914 248.95218048095705
calculate the region 3 average
min, max, mean
3.0004716 287.0032 114.463875
5%, 25%, 50%, 75%, 95%
20.31482172012329 69.79415512084961 119.05281829833984 160.53115463256836 194.18679809570312
